In [1]:
import datetime

import colormaps
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import xarray as xr
from jetutils.anyspell import get_persistent_jet_spells, mask_from_spells_pl, subset_around_onset, make_daily
from jetutils.clustering import Experiment
from jetutils.data import *
from jetutils.definitions import *
from jetutils.jet_finding import JetFindingExperiment, iterate_over_year_maybe_member, average_jet_categories, pers_from_cross_catd, spells_from_cross_catd_simple
from jetutils.plots import COLORS, Clusterplot
from jetutils.geospatial import *
from matplotlib.cm import ScalarMappable
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm

%load_ext IPython.extensions.autoreload
%autoreload 2
%matplotlib inline

IPython could not be loaded!


Found config override file at  /storage/homefs/hb22g102/.jetutils.ini
Guessed N_WORKERS :  4
Guessed MEMORY_LIMIT :  153600


In [ ]:
import xarray as xr

ds = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/uv/6H/results/Eddy_uv_natl_10days.zarr")

In [3]:
ds

<xarray.Dataset> Size: 87GB
Dimensions:  (time: 93504, lev: 2, lat: 181, lon: 321)
Coordinates:
  * time     (time) datetime64[ns] 748kB 1959-01-01 ... 2022-12-31T18:00:00
  * lev      (lev) float64 16B 250.0 850.0
  * lat      (lat) float64 1kB 0.0 0.5 1.0 1.5 2.0 ... 88.0 88.5 89.0 89.5 90.0
  * lon      (lon) float64 3kB -100.0 -99.5 -99.0 -98.5 ... 58.5 59.0 59.5 60.0
Data variables:
    up       (time, lev, lat, lon) float32 43GB dask.array<chunksize=(1390, 2, 72, 161), meta=np.ndarray>
    vp       (time, lev, lat, lon) float32 43GB dask.array<chunksize=(1390, 2, 72, 161), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-11T14:31 GRIB to CDM+CF via cfgrib-0.9.1...
    orig_lev:                [250, 850]

# EKE ?

In [38]:
uv = open_da(
    "ERA5",
    "plev",
    ("uv", ["u", "v"]),
    "6H",
    (1959, 2023),
    None,
    -100,
    60,
    0, 
    90,
    [250, 850]
)

In [39]:
import numpy as np
import dask.array as da

def convolve(in1, in2, mode="full", method="fft", axes=None):
    from scipy.signal import fftconvolve, oaconvolve
    from scipy.signal._signaltools import _init_freq_conv_axes


    in1 = da.asarray(in1)
    in2 = np.asarray(in2)

    # Checking for trivial cases and incorrect flags
    if in1.ndim == in2.ndim == 0:  # scalar inputs
        return in1 * in2
    elif in1.ndim != in2.ndim:
        raise ValueError("in1 and in2 should have the same dimensionality")
    elif in1.size == 0 or in2.size == 0:  # empty arrays
        return np.array([])

    if mode != "full" and mode != "same" and mode != "valid" and mode != "periodic":
        raise ValueError(
            "acceptable mode flags are 'valid'," " 'same', 'full' or 'periodic'"
        )
    if method not in ["fft", "oa"]:
        raise ValueError("acceptable method flags are 'oa', or 'fft'")

    # Pre-formatting or the the inputs, mainly for the `axes` argument
    in1, in2, axes = _init_freq_conv_axes(in1, in2, mode, axes, sorted_axes=False)

    # _init_freq_conv_axes calls a function that will swap out inputs if required
    # when mode == "valid".We want to avoid having in2 be a dask array thus check
    # to see if the inputs were swapped and raise an error.
    if type(in1) == np.ndarray:
        raise ValueError(
            "For 'valid' mode in1 has to be at least as large as in2 in every dimension"
        )

    s1 = in1.shape
    s2 = in2.shape

    # If all axe were removed by the preformatting we only have to rely
    # on multiplication broadcasting rules.
    if not len(axes):
        in_cv = in1 * in2
        # This is the "full" output that is also valid.
        # To get the "same" output we need to center in some dimensions.
        if mode == "same" or mode == "periodic":
            not_axes_but_s1_1 = [
                a
                for a in range(in1.ndim)
                if a not in axes and s1[a] == 1 and s2[a] != 1
            ]
            in_cv = in_cv[
                tuple(
                    slice((s2[a] - 1) // 2, (s2[a] - 1) // 2 + 1)
                    if a in not_axes_but_s1_1
                    else slice(None, None)
                    for a in range(in1.ndim)
                )
            ]
            return in_cv

    else:
        # This is kind of a hack but it works. 
        not_axes_but_s1_1 = [
            a for a in range(in1.ndim) if a not in axes and s1[a] == 1 and s2[a] != 1
        ]
        if len(not_axes_but_s1_1) and (mode == "full" or mode == "valid"):
            new_shape = tuple(
                s1[i] for i in range(in1.ndim) if i not in not_axes_but_s1_1
            )
            in1 = in1.reshape(new_shape)
            for a in not_axes_but_s1_1:
                in1 = da.stack([in1] * s2[a], axis=a)
            return convolve(in1, in2, mode=mode, method=method, axes=axes)

        # Deals with the case where there is at least one axis a in which we do not
        # do the convolution that has s2[a] == s1[a] != 1
        not_axes_but_same_shape = [
            a for a in range(in1.ndim) if a not in axes and s1[a] == s2[a] != 1
        ]
        if len(not_axes_but_same_shape):
            to_rechunk = [a for a in not_axes_but_same_shape if len(in1.chunks[a]) != 1]
            new_chunk_size = tuple(
                -1 if a in to_rechunk else "auto" for a in range(in1.ndim)
            )
            in1 = in1.rechunk(new_chunk_size)

        depth = {i: s2[i] // 2 for i in axes}

        # Flags even dimensions and removes them by adding zeros
        # This is done to avoid from having some results show up twice
        # at the edge of blocks
        even_flag = np.r_[[1 - s2[a] % 2 if a in axes else 0 for a in range(in1.ndim)]]
        target_shape = np.asarray(s2)
        target_shape += even_flag

        if any(target_shape != np.asarray(s2)):
            # padding axes where in2 is even
            pad_width = tuple(
                (even_flag[a], 0) if a in axes else (0, 0) for a in range(in1.ndim)
            )
            in2 = da.pad(in2, pad_width)

        if mode != "valid":
            pad_width = tuple(
                (depth[i] - even_flag[i], depth[i]) if i in axes else (0, 0)
                for i in range(in1.ndim)
            )
            in1 = da.pad(in1, pad_width)

        if mode == "periodic":
            boundary = "periodic"
        else:
            boundary = 0

        cv_dict = {"oa": oaconvolve, "fft": fftconvolve}

        cv_func = lambda x: cv_dict[method](x, in2, mode="same", axes=axes)

        complex_result = in1.dtype.kind == "c" or in2.dtype.kind == "c"

        if complex_result:
            dtype = "complex"
        else:
            dtype = "float"

        # Actualy does the convolution with all the parameters preformatted
        in_cv = in1.map_overlap(
            cv_func, depth=depth, boundary=boundary, trim=True, dtype=dtype
        )

        # The output as to be reduced depending on the `mode` argument
        if mode == "valid":
            output_slicing = tuple(
                slice(depth[i], s1[i] - (depth[i] - even_flag[i]), 1)
                if i in depth.keys()
                else slice(0, None)
                for i in range(in1.ndim)
            )
            in_cv = in_cv[output_slicing]

        elif mode != "full":
            # Only have to undo the padding
            output_slicing = tuple(
                slice(p[0], -p[1]) if p != (0, 0) else slice(0, None) for p in pad_width
            )
            in_cv = in_cv[output_slicing]

    return in_cv


In [40]:
from scipy.signal.windows import lanczos

half_len = 20
l_win = lanczos(2 * half_len + 1)[:, None, None, None]
dims = uv.dims
uv["ubar"] = (dims, (convolve(uv["u"].data, l_win)[half_len:-half_len] / l_win.sum()).astype(np.float32))
uv["up"] = uv["u"] - uv["ubar"]
uv["vbar"] = (dims, (convolve(uv["v"].data, l_win)[half_len:-half_len] / l_win.sum()).astype(np.float32))
uv["vp"] = uv["v"] - uv["vbar"]

del uv["ubar"]
del uv["vbar"]
del uv["u"]
del uv["v"]
uv = uv.chunk({"time": 1390, "lev": 2, "lat": 72, "lon": 161})

basepath = Path("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/uv/6H/results")

res = uv.to_zarr(basepath.joinpath("Eddy_uv_natl_10days.zarr"), compute=False)
compute(res, progress=True)

Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.


# 2023-24 wind data

In [4]:
basepath = Path(DATADIR, "ERA5/plev/high_wind/6H")
for year in [2024]:
    to_concat = []
    yearstr = str(year).zfill(4)
    for month in range(1, 13):
        monthstr = str(month).zfill(2)
        ds = standardize(xr.open_dataset(basepath.joinpath(f"{yearstr}{monthstr}_raw.nc")))
        ds["t"] = standardize(xr.open_dataarray(basepath.joinpath(f"{yearstr}{monthstr}_raw_t.nc")))
        opath = basepath.joinpath()
        ds["s"] = np.sqrt(ds["u"] ** 2 + ds["v"] ** 2)
        ds = flatten_by(ds, "s")
        ds["theta"] = ds["t"] * (1000 / ds["lev"]) ** KAPPA
        ds = ds.drop_vars("t")
        to_concat.append(ds)
    to_concat = xr.concat(to_concat, dim="time")
    to_concat.to_netcdf(basepath.joinpath(f"{yearstr}.nc"))

[########################################] | 100% Completed | 10.32 s
[########################################] | 100% Completed | 9.39 ss
[########################################] | 100% Completed | 11.00 s
[########################################] | 100% Completed | 9.99 ss
[########################################] | 100% Completed | 10.57 s
[########################################] | 100% Completed | 9.66 ss
[########################################] | 100% Completed | 10.57 s
[########################################] | 100% Completed | 9.67 ss
[########################################] | 100% Completed | 10.48 s
[########################################] | 100% Completed | 10.07 s
[########################################] | 100% Completed | 9.86 ss
[########################################] | 100% Completed | 10.39 s


In [31]:
ds.drop_vars("t")

<xarray.Dataset> Size: 323MB
Dimensions:  (time: 124, lat: 181, lon: 720)
Coordinates:
  * time     (time) datetime64[ns] 992B 2023-01-01 ... 2023-01-31T18:00:00
  * lat      (lat) float64 1kB 0.0 0.5 1.0 1.5 2.0 ... 88.0 88.5 89.0 89.5 90.0
  * lon      (lon) float64 6kB -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables:
    lev      (time, lat, lon) float32 65MB 350.0 350.0 350.0 ... 250.0 250.0
    u        (time, lat, lon) float32 65MB 9.997 10.62 ... -0.001694 -0.001694
    v        (time, lat, lon) float32 65MB 3.938 4.041 ... -4.578e-05 -4.578e-05
    s        (time, lat, lon) float32 65MB 10.74 11.36 ... 0.001694 0.001694
    theta    (time, lat, lon) float32 65MB 341.5 341.5 341.3 ... 312.9 312.9
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-20T09:04 GRIB to CDM+CF via cfgrib-0.9.1...
    orig_lev:                [175. 200. 225. 250. 300. 350.]
    flattened:               1

In [36]:
xr.open_dataset(basepath.joinpath("2022.nc"))

<xarray.Dataset> Size: 9GB
Dimensions:  (time: 1460, lat: 361, lon: 720)
Coordinates:
  * time     (time) datetime64[ns] 12kB 2022-01-01 ... 2022-12-31T18:00:00
  * lat      (lat) float32 1kB -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 3kB -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables:
    s        (time, lat, lon) float32 2GB ...
    lev      (time, lat, lon) float32 2GB ...
    u        (time, lat, lon) float32 2GB ...
    v        (time, lat, lon) float32 2GB ...
    theta    (time, lat, lon) float64 3GB ...
Attributes:
    orig_lev:   [175 200 225 250 300 350]
    flattened:  1

# new pvs

In [ ]:
from wavebreaking import to_xarray as to_xarray_orig
import geopandas as gpd
from jetutils.definitions import *
from tqdm import tqdm

all_events = {}
basepath = Path(DATADIR, "ERA5/RWB_index/pv")
levs = list(range(310, 365, 5))
for level in levs:
    events = gpd.read_parquet(basepath.joinpath(f"era5_pv_streamers_{level}K_1959-2022.parquet"))
    stratospheric = events[events.mean_var >= 2]
    tropospheric = events[events.mean_var < 2]
    
    all_events[f"SAPVS_{level}"] = stratospheric[stratospheric.intensity >= 0]
    all_events[f"TAPVS_{level}"] = tropospheric[tropospheric.intensity >= 0]
    all_events[f"SCPVS_{level}"] = stratospheric[stratospheric.intensity < 0]
    all_events[f"TCPVS_{level}"] = tropospheric[tropospheric.intensity < 0]
    
    events = gpd.read_parquet(basepath.joinpath(f"era5_pv_overturnings_{level}K_1959-2022.parquet"))
    
    all_events[f"APVO_{level}"] = events[events.orientation == "anticyclonic"]
    all_events[f"CPVO_{level}"] = events[events.orientation == "cyclonic"]
    
    
varname = "flag"
dtype = {"flag": np.uint32, "intensity": np.float32, "mean_var": np.float32, "event_area": np.float32}[varname]
coords = {
    "time": TIMERANGE,
    "lat": np.arange(0, 90.5, 1),
    "lon": np.arange(-100, 60.5, 1),
}
shape = [len(co) for co in coords.values()]
dummy_da = xr.DataArray(np.zeros(shape, dtype=dtype), coords=coords)

basepath = Path(DATADIR, "ERA5/thetalev")
sublevels = list(range(310, 355, 5))
for name in ["SAPVS", "TAPVS", "SCPVS", "TCPVS", "APVO", "CPVO"]:
    da = []
    for lev in tqdm(levs):
        da.append(to_xarray_orig(dummy_da, all_events[f"{name}_{lev}"]), flag=varname)
    da = xr.concat(da, dim="lev").assign_coords(lev=levs)
    basepath.joinpath(f"{name}/6H").mkdir(parents=True, exist_ok=True)
    da.to_netcdf(basepath.joinpath(f"{name}/6H/full.nc"))
    da = da.sel(lev=sublevels).any("lev").resample(time="1D").any().astype(np.uint8)
    basepath.joinpath(f"{name}/dailyany").mkdir(parents=True, exist_ok=True)
    da.to_netcdf(basepath.joinpath(f"{name}/dailyany/full.nc"))

In [2]:
xr.open_dataarray("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/thetalev/SAPVS/6H/full.nc")

<xarray.DataArray 'flag' (lev: 11, time: 93504, lat: 101, lon: 161)> Size: 17GB
[16725153984 values with dtype=int8]
Coordinates:
  * lev      (lev) int64 88B 310 315 320 325 330 335 340 345 350 355 360
  * time     (time) datetime64[ns] 748kB 1959-01-01 ... 2022-12-31T18:00:00
  * lat      (lat) float64 808B 0.0 1.0 2.0 3.0 4.0 ... 97.0 98.0 99.0 100.0
  * lon      (lon) float64 1kB -100.0 -99.0 -98.0 -97.0 ... 57.0 58.0 59.0 60.0
Attributes:
    long_name:  flag wave breaking

In [ ]:
from itertools import product
import shutil
streamers_subtypes = ["_".join(both) for both in product(["stratospheric", "tropospheric"], ["anticyclonic", "cyclonic"])]
types = {
    "overturnings": ["anticyclonic", "cyclonic"], 
    "streamers": streamers_subtypes
}
basepath = Path(DATADIR, "ERA5/thetalev")
for type_, subtypes in types.items():
    for subtype in subtypes:
        if "_" in subtype:
            shorthand_1 = "".join([sub[0].upper() for sub in subtype.split("_")])
            shorthand_2 = subtype.split("_")
            shorthand_2 = shorthand_2[1][:4] + "_" + shorthand_2[0].rstrip("spheric")
        else:
            shorthand_1 = subtype[0].upper()
            shorthand_2 = subtype[:4]
        file_stem = f"{type_}_{shorthand_2}"
        shorthand = shorthand_1 + "PV" + type_[0].upper()
        this_path = basepath.joinpath(shorthand)
        this_path.mkdir(exist_ok=True)
        for freq in ["6H", "dailyany"]:
            file_spec = "" if freq == "6H" else f"_{freq}"
            file_stem_ = f"{type_}_{shorthand_2}_natl{file_spec}.nc"
            this_path_ = this_path.joinpath(freq)
            this_path_.mkdir(exist_ok=True)
            source = Path(DATADIR, "ERA5", "RWB_index", "pv").joinpath(f"{file_stem_}")
            dest = this_path_.joinpath("full.nc")
            print(source, dest)
            shutil.copy(source, dest)
        # print(this_path, shorthand_2)
    

In [ ]:
basepath = Path(DATADIR, "ERA5/RWB_index/pv")
files_to_treat = basepath.glob("*.nc")
levels = list(range(310, 355, 5))
for f in tqdm(files_to_treat):
    da = xr.open_dataarray(f)
    da = da.sel(lev=levels).any("lev").resample(time="1D").any().astype(np.uint8)
    da.to_netcdf(f.parent.joinpath(f"{f.stem}_dailyany.nc"))

6it [00:45,  7.53s/it]


In [28]:
from jetutils.data import *
basepath = Path(DATADIR, "ERA5/RWB_index/pv")
files_to_treat = basepath.glob("*dailyany.nc")
for f in tqdm(files_to_treat):
    da = xr.open_dataarray(f)
    clim = smooth(compute_clim(da, "dayofyear"), {"dayofyear": ("win", 15)}).astype(np.float32)
    anom = da.astype(np.float32).groupby("time.dayofyear") - clim
    clim.astype(np.float32).to_netcdf(f.parent.joinpath(f"{f.stem}_clim.nc"))
    anom.astype(np.float32).to_netcdf(f.parent.joinpath(f"{f.stem}_anom.nc"))

6it [00:17,  2.86s/it]


In [ ]:
for rwb_type in ["APVO", "CPVO", "SAPVS", "SCPVS", "TAPVS", "TCPVS"]:
    dh = DataHandler.from_specs("ERA5", "thetalev", "APVO", "dailyany", "all", None, -80, 40, 15, 80)
    da = compute(dh.da)
    

[########################################] | 100% Completed | 706.27 ms


# CESM clims

In [6]:
da_tp = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/past.zarr")

clim = da_tp.groupby("time.dayofyear").mean()
clim = smooth(clim, {'dayofyear': ('win', 15)})
clim = compute(clim, progress_flag=True)
clim.to_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/past_clim.zarr")

[########################################] | 100% Completed | 126.37 s


In [ ]:
anom = da_tp.groupby("time.dayofyear") - clim
anom = compute(anom, progress_flag=True)
anom.to_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/past_anom.zarr")

In [ ]:
da_T = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/TS/past.zarr")

clim = da_tp.groupby("time.dayofyear").mean()
clim = smooth(clim, {'dayofyear': ('win', 15)})
clim = compute(clim, progress_flag=True)
clim.to_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/past_clim.zarr")

In [7]:
# da_tp = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/future.zarr")

# clim = da_tp.groupby("time.dayofyear").mean()
# clim = smooth(clim, {'dayofyear': ('win', 15)})
# clim = compute(clim, progress_flag=True)
# clim.to_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/PRECL/future_clim.zarr")

# create jet relative climatologies

In [2]:
all_times = (
    pl.datetime_range(
        start=pl.datetime(1959, 1, 1),
        end=pl.datetime(2023, 1, 1),
        closed="left",
        interval="6h",
        eager=True,
        time_unit="ms",
    )
    .rename("time")
    .to_frame()
)
summer_filter = (
    all_times
    .filter(pl.col("time").dt.month().is_in([6, 7, 8, 9]))
    .filter(pl.col("time").dt.ordinal_day() > 166)
)
summer = summer_filter["time"]
summer_daily = summer.filter(summer.dt.hour() == 0)
big_summer = all_times.filter(pl.col("time").dt.month().is_in([6, 7, 8, 9]))
big_summer_daily = big_summer.filter(big_summer["time"].dt.hour() == 0)

dh = DataHandler("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/high_wind/6H/results/8")
exp = JetFindingExperiment(dh)
ds = exp.ds
all_jets_one_df = exp.find_jets(force=False, base_s_thresh=0.55, hole_size=6)
all_jets_one_df = exp.categorize_jets(None, ["s", "theta"], force=False, n_init=10, init_params="k-means++", mode="week").cast({"time": pl.Datetime("ms")})
props_uncat = exp.props_as_df(False).cast({"time": pl.Datetime("ms")})

props_as_df = average_jet_categories(props_uncat, polar_cutoff=0.5)

props_summer = summer_filter.join(props_as_df, on="time")
phat_filter = (pl.col("is_polar") < 0.5) | ((pl.col("is_polar") > 0.5) & (pl.col("int") > 1.3e8))

phat_jets = all_jets_one_df.filter((pl.col("is_polar").mean().over(["time", "jet ID"]) < 0.5) | ((pl.col("is_polar").mean().over(["time", "jet ID"]) > 0.5) & (pl.col("int").mode().first().over(["time", "jet ID"]) > 1.3e8)))
phat_jets_catd = phat_jets.with_columns(**{"jet ID": (pl.col("is_polar").mean().over(["time", "jet ID"]) > 0.5).cast(pl.UInt32())})
phat_props = props_uncat.filter(phat_filter)
phat_props_catd = average_jet_categories(phat_props, polar_cutoff=0.5)

phat_props_catd = phat_props_catd.join(phat_props_catd.rolling("time", period="2d", group_by="jet").agg(**{f"{col}_var": pl.col(col).var() for col in ["mean_lon", "mean_lat", "mean_s", "s_star"]}), on=["time", "jet"])

phat_props_catd_summer = summer_filter.join(phat_props_catd, on="time")

cross_catd_ofile = exp.path.joinpath("cross_catd.parquet")
if cross_catd_ofile.is_file():
    cross_catd = pl.read_parquet(cross_catd_ofile)
else:
    cross_catd = track_jets(phat_jets_catd)
    cross_catd.write_parquet(cross_catd_ofile)

pers = pers_from_cross_catd(cross_catd)
spells_list = spells_from_cross_catd_simple(cross_catd, season=summer, q_STJ=0.705, q_EDJ=0.836, minlen=datetime.timedelta(days=5), smooth=datetime.timedelta(hours=24), fill_holes=datetime.timedelta(hours=18))

daily_spells_list = {a: make_daily(b, "spell", ["len", "spell_of"]) for a, b in spells_list.items()}

In [5]:
def compute_emf_2d_conv(ds):
    lon, lat = ds["lon"].values, ds["lat"].values
    xlon, ylat = np.meshgrid(lon, lat)

    _, dlonx = np.gradient(xlon)
    dlaty, _ = np.gradient(ylat)

    dx = RADIUS * np.radians(dlaty) * degcos(ylat)
    dy = RADIUS * np.radians(dlaty)
    
    e1 = 0.5 * (ds["vp"] ** 2 - ds["up"] ** 2)
    e2 = - ds["up"] * ds["vp"]
    de1dx = ds["up"].copy(data=np.gradient(e1, axis=2)) / dx[None, :, :]
    de2dy = ds["up"].copy(data=np.gradient(e2, axis=1)) / dy[None, :, :]
    return de1dx + de2dy

In [ ]:
args = ["all", None, -100, 60, 0, 90]

da_T = open_da("ERA5", "surf", "t2m", "dailymean", *args)
da_T = compute(da_T)
create_jet_relative_dataset(phat_jets_catd, exp.path, da_T, suffix="_phat_catd")
del da_T

da_tp = open_da("ERA5", "surf", "tp", "dailysum", *args)
da_tp = compute(da_tp)
create_jet_relative_dataset(phat_jets_catd, exp.path, da_tp, suffix="_phat_catd")
del da_tp

da_pv = open_da("ERA5", "thetalev", "PV330", "dailymean", *args)
da_pv = compute(da_pv).rename("PV330")
create_jet_relative_dataset(phat_jets_catd, exp.path, da_pv, suffix="_phat_catd")
del da_pv

da_pv = open_da("ERA5", "thetalev", "PV350", "dailymean", *args)
da_pv = compute(da_pv)
create_jet_relative_dataset(phat_jets_catd, exp.path, da_pv, suffix="_phat_catd")
del da_pv

varnames_rwb = ["APVO", "CPVO"]
for var in varnames_rwb:
    da_rwb = open_da("ERA5", "thetalev", var, "dailyany", *args)
    da_rwb = compute(da_rwb)
    create_jet_relative_dataset(phat_jets_catd, exp.path, da_rwb, suffix="_phat_catd")
    del da_rwb

da_theta2pvu = open_da("ERA5", "surf", "alot2pvu", "dailymean", *args)
da_theta2pvu = compute(da_theta2pvu)
create_jet_relative_dataset(phat_jets_catd, exp.path, da_theta2pvu, suffix="_phat_catd")
del da_theta2pvu

ds = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/uv/6H/results/Eddy_uv_natl_6days.zarr")

EKE = ds["EKE"].sel(lev=250).rename("EKE250")
EKE = EKE.resample(time="1d").mean()
EKE = compute(EKE, progress_flag=True)
create_jet_relative_dataset(phat_jets_catd, exp.path, EKE, suffix="_phat_catd")
del EKE

EMF = ds["EMF"].sel(lev=250).rename("EMF250")
EMF = compute(EMF, progress_flag=True)
EMF = EMF.resample(time="1d").mean()
create_jet_relative_dataset(phat_jets_catd, exp.path, EMF, suffix="_phat_catd")
del EMF

up = ds["up"].sel(lev=250).rename("up250")
up = up.resample(time="1d").mean()
up = compute(up, progress_flag=True)
create_jet_relative_dataset(phat_jets_catd, exp.path, up, suffix="_phat_catd")
del up

vp = ds["vp"].sel(lev=250).rename("vp250")
vp = compute(vp, progress_flag=True)
vp = vp.resample(time="1d").mean()
create_jet_relative_dataset(phat_jets_catd, exp.path, vp, suffix="_phat_catd")
del vp

In [ ]:
args = ["all", None, -100, 60, 0, 90]

# da_T = open_da("ERA5", "surf", "t2m", "dailymean", *args)
# da_T = compute(da_T)
# create_jet_relative_dataset(phat_jets_catd, exp.path, da_T, suffix="_phat_catd")
# del da_T

# da_tp = open_da("ERA5", "surf", "tp", "dailysum", *args)
# da_tp = compute(da_tp)
# create_jet_relative_dataset(phat_jets_catd, exp.path, da_tp, suffix="_phat_catd")
# del da_tp

# da_pv = open_da("ERA5", "thetalev", "PV330", "dailymean", *args)
# da_pv = compute(da_pv).rename("PV330")
# create_jet_relative_dataset(phat_jets_catd, exp.path, da_pv, suffix="_phat_catd")
# del da_pv

# da_pv = open_da("ERA5", "thetalev", "PV350", "dailymean", *args)
# da_pv = compute(da_pv)
# create_jet_relative_dataset(phat_jets_catd, exp.path, da_pv, suffix="_phat_catd")
# del da_pv

# varnames_rwb = ["APVO", "CPVO"]
# for var in varnames_rwb:
#     da_rwb = open_da("ERA5", "thetalev", var, "dailyany", *args)
#     da_rwb = compute(da_rwb)
#     create_jet_relative_dataset(phat_jets_catd, exp.path, da_rwb, suffix="_phat_catd")
#     del da_rwb

# da_theta2pvu = open_da("ERA5", "surf", ("alot2pvu", "theta"), "dailymean", *args)
# da_theta2pvu = compute(da_theta2pvu)
# create_jet_relative_dataset(phat_jets_catd, exp.path, da_theta2pvu, suffix="_phat_catd")
# del da_theta2pvu

ds = xr.open_zarr("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/uv/6H/results/Eddy_uv_natl_10days.zarr")

# up = ds["up"].sel(lev=250).rename("up250")
# up = up.resample(time="1d").mean()
# up = compute(up, progress_flag=True)
# create_jet_relative_dataset(phat_jets_catd, exp.path, up, suffix="_phat_catd")
# del up

# vp = ds["vp"].sel(lev=250).rename("vp250")
# vp = compute(vp, progress_flag=True)
# vp = vp.resample(time="1d").mean()
# create_jet_relative_dataset(phat_jets_catd, exp.path, vp, suffix="_phat_catd")
# del vp

# EMFconv = compute_emf_2d_conv(ds.sel(lev=250)).rename("EMFconv250")
# EMFconv = compute(EMFconv, progress_flag=True)
# EMFconv = EMFconv.resample(time="1d").mean()
# create_jet_relative_dataset(phat_jets_catd, exp.path, EMFconv, suffix="_phat_catd")
# del EMFconv

EKE = (ds.sel(lev=250)["up"] ** 2 + ds.sel(lev=250)["vp"] ** 2) * 0.5
EKE = compute(EKE, progress_flag=True).rename("EKE250")
EKE = EKE.resample(time="1d").mean()
create_jet_relative_dataset(phat_jets_catd, exp.path, EKE, suffix="_phat_catd")
del EKE

[########################                ] | 62% Completed | 16.71 ss

In [6]:
from jetutils.data import compute_all_dailymeans
compute_all_dailymeans("ERA5", "surf", "alot2pvu")

  0%|          | 0/64 [00:00<?, ?it/s]

 98%|█████████▊| 63/64 [18:37<00:17, 17.74s/it]


In [2]:
from pathlib import Path
base_path_1 = Path(f"{DATADIR}/ERA5/surf/theta2PVU/6H")
base_path_2 = Path(f"{DATADIR}/ERA5/surf/theta2PVU/dailymean")
# base_path_2.mkdir()
for year in YEARS:
    print(year, end="\r")
    opath_1 = base_path_1.joinpath(f"{year}.nc")
    opath_2 = base_path_2.joinpath(f"{year}.nc")

    if opath_2.is_file():
        continue
    
    this_pv = standardize(open_dataarray(opath_1))
    this_pv = compute(this_pv, progress_flag=False)
    this_pv = this_pv.resample(time="1d").mean()    
    this_pv.to_netcdf(opath_2)

# arco-era5 tests

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=200,
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full).chunk("auto")

from pathlib import Path
base_path_1 = Path(f"{DATADIR}/ERA5/plev/t200/6H")
base_path_2 = Path(f"{DATADIR}/ERA5/plev/t200/dailymean")
# base_path_1.mkdir(parents=True)
# base_path_2.mkdir(parents=True)
for year in YEARS:
    print(year)
    opath_1 = base_path_1.joinpath(f"{year}.nc")
    opath_2 = base_path_2.joinpath(f"{year}.nc")

    if opath_2.is_file():
        continue
    this_temp = temp_full.sel(time=temp_full.time.dt.year == year)
    this_temp = this_temp.reset_coords("lev", drop=True)
    this_temp = compute(this_temp, progress_flag=True)
    this_temp.to_netcdf(opath_1)
    
    this_temp = this_temp.resample(time="1d").mean()
    this_temp.to_netcdf(opath_2)

In [5]:
temp_full

<xarray.DataArray 'temperature' (time: 124424, lat: 181, lon: 720)> Size: 65GB
[16214935680 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 724B 0.0 0.5 1.0 1.5 2.0 ... 88.0 88.5 89.0 89.5 90.0
  * time     (time) datetime64[ns] 995kB 1940-01-01 ... 2025-02-28T18:00:00
  * lon      (lon) float32 3kB -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
    lev      int64 8B 200
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full)

orig_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean")
base_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean_2")
for year in tqdm(YEARS):
    for month in trange(1, 13, leave=False):
        month_str = str(month).zfill(2)
        opath = base_path.joinpath(f"{year}{month_str}.nc")
        if opath.is_file():
            continue
        ipath = orig_path.joinpath(f"{year}{month_str}.nc")
        ds = xr.open_dataset(ipath)
        this_temp = temp_full.sel(time=ds.time.values, lev=ds["lev"])
        this_temp = this_temp * (1000 / this_temp.lev) ** KAPPA
        this_temp = this_temp.reset_coords("lev", drop=True)
        ds["theta"] = compute(this_temp, progress_flag=True)
        ds.to_netcdf(opath)

# CESM

### new download with urls

## newnew merger script: download then postprocess:

In [45]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from jetutils.definitions import DATADIR, KAPPA, compute
from jetutils.data import standardize, flatten_by, extract
import intake
import numpy as np
import xarray as xr
from pathlib import Path
from dask.diagnostics import ProgressBar

varname = "PRECL"
component = "atm" # for land variables like RAIN, "atm" for atmospheric variables like wind, and "ocn" for ocean variables
forcing_variant = "cmip6" # other option is "smbb", which stands for "SMoothed Biomass Burning"
out_path = Path(DATADIR, "CESM2", varname)
minlon, maxlon, minlat, maxlat = None, None, 0, 90
levels = None
years = {
    "past": np.arange(1970, 2010),
    "future": np.arange(2060, 2100),
}

col_url = (
    "https://ncar-cesm2-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm2-le.json"
)
catalog = intake.open_esm_datastore(col_url)

catalog_subset = catalog.search(variable=varname, frequency='daily', forcing_variant=forcing_variant)
dsets = catalog_subset.to_dataset_dict(storage_options={'anon':True})

ds_past = dsets[f"{component}.historical.daily.{forcing_variant}"]
ds_future = dsets[f"{component}.ssp370.daily.{forcing_variant}"]

ds_past_ns = (
    standardize(ds_past)
    .reset_coords("time_bnds", drop=True)
    .squeeze()
    .isel(time=np.isin(ds_past.time.dt.year, years["past"]))
    .sel(lon=slice(minlon, maxlon))
    .sel(lat=slice(minlat, maxlat))
)
ds_future_ns = (
    standardize(ds_future)
    .reset_coords("time_bnds", drop=True)
    .squeeze()
    .isel(time=np.isin(ds_future.time.dt.year, years["future"]))
    .sel(lon=slice(minlon, maxlon))
    .sel(lat=slice(minlat, maxlat))
)
if levels is not None and "lev" in ds_past_ns.dims:
    ds_past_ns = ds_past_ns.isel(lev=levels)
    ds_future_ns = ds_future_ns.isel(lev=levels)

opath = out_path.joinpath("historical")
opath.mkdir(parents=True, exist_ok=True)
for varname in ds_past_ns.data_vars:
    ds_past_ns[varname] = ds_past_ns[varname].drop_encoding()
saved = ds_past_ns.to_zarr(opath.joinpath("ds.zarr"), compute=False, mode="w")
with ProgressBar():
    saved.compute()
    
opath = out_path.joinpath("ssp370")
opath.mkdir(parents=True, exist_ok=True)
for varname in ds_future_ns.data_vars:
    ds_future_ns[varname] = ds_future_ns[varname].drop_encoding()
saved = ds_future_ns.to_zarr(opath.joinpath("ds.zarr"), compute=False, mode="w")
with ProgressBar():
    saved.compute()
#     ds_past_ns = ds_past_ns.load()
# ds_past_ns.to_netcdf(out_path.joinpath(out_name_past))
# del ds_past_ns # free up memory

# with ProgressBar():
#     ds_future_ns = ds_future_ns.load()
# ds_future_ns.to_netcdf(out_path.joinpath(out_name_future))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


[##################                      ] | 45% Completed | 10m 57ss


KeyboardInterrupt: 

### new cesm zarrification

In [3]:
basepath = Path("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/high_wind/ssp370")
paths = list(basepath.glob("*.nc"))
names = [path.stem.split("-") for path in paths]
members = [name[0] for name in names]
years = [name[1] for name in names]
for i, member in enumerate(tqdm(np.unique(members))):
    da = xr.open_mfdataset(basepath.joinpath(f"{member}-*.nc").as_posix())
    kwargs = {"mode": "w"} if i == 0 else {"mode": "a", "append_dim": "member"}
    da["member"] = da["member"].astype("<U15")
    da = da.expand_dims("member").copy(deep=True)
    break
    # da.to_zarr(basepath.joinpath("ds.zarr"), **kwargs)

  0%|          | 0/50 [00:03<?, ?it/s]
